# Tutorial 03: Systems of Linear Equations

Solving Ax = b - the foundation of ML optimization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')

## 1. Visualizing Systems of Equations (2D)

In [ ]:
def plot_2d_system(A, b, title="System of 2 Equations"):
    """Visualize a 2x2 system as intersecting lines."""
    fig, ax = plt.subplots(figsize=(10, 8))
    
    x_range = np.linspace(-5, 5, 100)
    
    # Each row of A gives a line: a1*x + a2*y = b
    # Rearrange: y = (b - a1*x) / a2
    
    colors = ['blue', 'red', 'green', 'orange']
    for i in range(A.shape[0]):
        a1, a2 = A[i, 0], A[i, 1]
        if abs(a2) > 1e-10:
            y = (b[i] - a1 * x_range) / a2
            ax.plot(x_range, y, colors[i % len(colors)], linewidth=2,
                   label=f'{a1}x + {a2}y = {b[i]}')
        else:
            # Vertical line
            ax.axvline(x=b[i]/a1, color=colors[i % len(colors)], linewidth=2,
                      label=f'{a1}x = {b[i]}')
    
    # Try to find and plot solution
    try:
        x_sol = np.linalg.solve(A, b)
        ax.plot(x_sol[0], x_sol[1], 'ko', markersize=15, zorder=5)
        ax.annotate(f'Solution: ({x_sol[0]:.2f}, {x_sol[1]:.2f})',
                   xy=(x_sol[0], x_sol[1]), xytext=(10, 10),
                   textcoords='offset points', fontsize=12)
    except:
        pass
    
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(title)
    ax.legend()
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    plt.show()

# Unique solution
A1 = np.array([[1, 1], [1, -1]])
b1 = np.array([4, 2])
plot_2d_system(A1, b1, "Unique Solution: Lines Intersect")

In [ ]:
# No solution (parallel lines)
A2 = np.array([[1, 2], [2, 4]])
b2 = np.array([3, 8])
plot_2d_system(A2, b2, "No Solution: Parallel Lines")

In [ ]:
# Infinitely many solutions (same line)
A3 = np.array([[1, 2], [2, 4]])
b3 = np.array([3, 6])
plot_2d_system(A3, b3, "Infinite Solutions: Same Line")

## 2. Gaussian Elimination

In [ ]:
def gaussian_elimination(A, b, verbose=True):
    """Solve Ax = b using Gaussian elimination with back substitution."""
    n = len(b)
    # Create augmented matrix
    M = np.hstack([A.astype(float), b.reshape(-1, 1).astype(float)])
    
    if verbose:
        print("Initial augmented matrix:")
        print(M)
        print()
    
    # Forward elimination
    for i in range(n):
        # Find pivot
        max_row = i + np.argmax(np.abs(M[i:, i]))
        M[[i, max_row]] = M[[max_row, i]]
        
        if np.abs(M[i, i]) < 1e-10:
            print("Matrix is singular!")
            return None
        
        # Eliminate below
        for j in range(i + 1, n):
            factor = M[j, i] / M[i, i]
            M[j, i:] -= factor * M[i, i:]
        
        if verbose:
            print(f"After eliminating column {i}:")
            print(M)
            print()
    
    # Back substitution
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (M[i, -1] - np.dot(M[i, i+1:n], x[i+1:])) / M[i, i]
    
    return x

# Test
A = np.array([[1, 2, 1], [2, 1, 1], [3, 1, 2]])
b = np.array([9, 8, 13])

x = gaussian_elimination(A, b)
print(f"Solution: x = {x}")
print(f"Verification Ax = {A @ x}")
print(f"Expected b = {b}")

## 3. LU Decomposition

In [ ]:
def lu_decomposition(A):
    """Compute LU decomposition (without pivoting)."""
    n = A.shape[0]
    L = np.eye(n)
    U = A.astype(float).copy()
    
    for i in range(n):
        for j in range(i + 1, n):
            factor = U[j, i] / U[i, i]
            L[j, i] = factor
            U[j, i:] -= factor * U[i, i:]
    
    return L, U

def solve_lu(L, U, b):
    """Solve Ax = b using LU decomposition."""
    n = len(b)
    
    # Forward substitution: Ly = b
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    
    # Back substitution: Ux = y
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    
    return x

# Test
A = np.array([[4, 3], [6, 3]], dtype=float)
b = np.array([10, 12], dtype=float)

L, U = lu_decomposition(A)
print("L =")
print(L)
print("\nU =")
print(U)
print("\nL @ U =")
print(L @ U)
print("\nOriginal A =")
print(A)

x = solve_lu(L, U, b)
print(f"\nSolution: x = {x}")

## 4. Least Squares Solution

In [ ]:
def visualize_least_squares():
    """Show overdetermined system and least squares solution."""
    np.random.seed(42)
    
    # Generate noisy linear data
    n_points = 50
    x = np.linspace(0, 10, n_points)
    true_slope, true_intercept = 2.5, 1
    y = true_slope * x + true_intercept + np.random.randn(n_points) * 2
    
    # Set up overdetermined system: A @ [w0, w1]^T = y
    A = np.column_stack([np.ones(n_points), x])
    
    # Solve via normal equations
    ATA = A.T @ A
    ATy = A.T @ y
    w = np.linalg.solve(ATA, ATy)
    
    print(f"Normal equations solution:")
    print(f"  Intercept: {w[0]:.4f}")
    print(f"  Slope: {w[1]:.4f}")
    
    # Compare with numpy's least squares
    w_numpy, residuals, rank, s = np.linalg.lstsq(A, y, rcond=None)
    print(f"\nNumPy lstsq solution: {w_numpy}")
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(x, y, alpha=0.6, label='Data points')
    
    x_line = np.linspace(0, 10, 100)
    y_line = w[0] + w[1] * x_line
    ax.plot(x_line, y_line, 'r-', linewidth=2, label=f'Least squares: y = {w[0]:.2f} + {w[1]:.2f}x')
    
    # Show residuals
    y_pred = A @ w
    for i in range(0, n_points, 5):
        ax.plot([x[i], x[i]], [y[i], y_pred[i]], 'g-', alpha=0.5)
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('Least Squares: Minimizing Sum of Squared Residuals')
    ax.legend()
    plt.show()
    
    return A, y, w

A, y, w = visualize_least_squares()

## 5. Geometric View: Projection onto Column Space

In [ ]:
def visualize_projection_3d():
    """Show least squares as projection onto column space."""
    from mpl_toolkits.mplot3d import Axes3D
    
    # Simple example: 3 equations, 2 unknowns
    A = np.array([[1, 0], [0, 1], [1, 1]])
    b = np.array([1, 1, 1])
    
    # Least squares solution
    x_ls = np.linalg.lstsq(A, b, rcond=None)[0]
    projection = A @ x_ls
    
    fig = plt.figure(figsize=(12, 5))
    
    # 3D plot
    ax1 = fig.add_subplot(121, projection='3d')
    
    # Column space (plane spanned by columns of A)
    col1, col2 = A[:, 0], A[:, 1]
    
    # Create mesh for plane
    s_range = np.linspace(-1, 2, 10)
    t_range = np.linspace(-1, 2, 10)
    S, T = np.meshgrid(s_range, t_range)
    
    X = S * col1[0] + T * col2[0]
    Y = S * col1[1] + T * col2[1]
    Z = S * col1[2] + T * col2[2]
    
    ax1.plot_surface(X, Y, Z, alpha=0.3, color='blue')
    
    # Plot b
    ax1.scatter([b[0]], [b[1]], [b[2]], color='red', s=100, label='b')
    
    # Plot projection
    ax1.scatter([projection[0]], [projection[1]], [projection[2]], 
                color='green', s=100, label='Ax (projection)')
    
    # Plot error vector
    ax1.plot([b[0], projection[0]], [b[1], projection[1]], [b[2], projection[2]],
            'r--', linewidth=2, label='Error (perpendicular)')
    
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_zlabel('z')
    ax1.set_title('Least Squares = Projection\nonto Column Space of A')
    ax1.legend()
    
    # 2D view showing residual is perpendicular
    ax2 = fig.add_subplot(122)
    
    error = b - projection
    
    # Verify error is perpendicular to column space
    print(f"Error · col1 = {np.dot(error, col1):.6f} (should be ~0)")
    print(f"Error · col2 = {np.dot(error, col2):.6f} (should be ~0)")
    
    ax2.bar(['Error · col1', 'Error · col2'], [np.dot(error, col1), np.dot(error, col2)])
    ax2.axhline(y=0, color='k', linestyle='--')
    ax2.set_ylabel('Dot Product')
    ax2.set_title('Error is Perpendicular to Column Space\n(Dot products ≈ 0)')
    
    plt.tight_layout()
    plt.show()

visualize_projection_3d()

## 6. Condition Number and Numerical Stability

In [ ]:
def demonstrate_condition_number():
    """Show how condition number affects solution stability."""
    
    # Well-conditioned matrix
    A_good = np.array([[1, 0], [0, 1]])
    
    # Ill-conditioned matrix (nearly singular)
    A_bad = np.array([[1, 1], [1, 1.0001]])
    
    b = np.array([1, 2])
    
    # Solve both
    x_good = np.linalg.solve(A_good, b)
    x_bad = np.linalg.solve(A_bad, b)
    
    print("Well-conditioned matrix:")
    print(f"  Condition number: {np.linalg.cond(A_good):.2f}")
    print(f"  Solution: {x_good}")
    
    print("\nIll-conditioned matrix:")
    print(f"  Condition number: {np.linalg.cond(A_bad):.2f}")
    print(f"  Solution: {x_bad}")
    
    # Show sensitivity to perturbation
    print("\n--- Sensitivity to small perturbation in b ---")
    
    delta_b = np.array([0.001, 0])
    
    x_good_perturbed = np.linalg.solve(A_good, b + delta_b)
    x_bad_perturbed = np.linalg.solve(A_bad, b + delta_b)
    
    print(f"\nPerturbation in b: {delta_b}")
    print(f"\nWell-conditioned:")
    print(f"  Change in x: {np.linalg.norm(x_good_perturbed - x_good):.6f}")
    print(f"\nIll-conditioned:")
    print(f"  Change in x: {np.linalg.norm(x_bad_perturbed - x_bad):.6f}")

demonstrate_condition_number()

## 7. Ridge Regression (Regularization)

In [ ]:
def compare_ols_ridge():
    """Compare ordinary least squares with ridge regression."""
    np.random.seed(42)
    
    # Generate data with collinear features (ill-conditioned)
    n_samples = 100
    x1 = np.random.randn(n_samples)
    x2 = x1 + np.random.randn(n_samples) * 0.01  # Almost same as x1
    X = np.column_stack([np.ones(n_samples), x1, x2])
    
    # True relationship only uses x1
    y = 1 + 2 * x1 + np.random.randn(n_samples) * 0.5
    
    print(f"Condition number of X^T X: {np.linalg.cond(X.T @ X):.2f}")
    
    # OLS solution (can be unstable)
    try:
        w_ols = np.linalg.solve(X.T @ X, X.T @ y)
        print(f"\nOLS weights: {w_ols}")
    except:
        print("OLS failed due to singular matrix")
        w_ols = None
    
    # Ridge regression for different lambda values
    lambdas = [0.001, 0.01, 0.1, 1, 10]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    w_history = []
    cond_history = []
    
    for lam in lambdas:
        # Ridge: (X^T X + λI) w = X^T y
        XTX_reg = X.T @ X + lam * np.eye(X.shape[1])
        w_ridge = np.linalg.solve(XTX_reg, X.T @ y)
        w_history.append(w_ridge)
        cond_history.append(np.linalg.cond(XTX_reg))
        print(f"\nλ = {lam}: weights = {w_ridge}, cond = {cond_history[-1]:.2f}")
    
    w_history = np.array(w_history)
    
    # Plot weight paths
    for i in range(w_history.shape[1]):
        ax.semilogx(lambdas, w_history[:, i], 'o-', label=f'w{i}')
    
    ax.set_xlabel('λ (regularization)')
    ax.set_ylabel('Weight value')
    ax.set_title('Ridge Regression: Weight Paths\n(Regularization stabilizes ill-conditioned problem)')
    ax.legend()
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    plt.show()

compare_ols_ridge()

## 8. Iterative Method: Gradient Descent

In [ ]:
def gradient_descent_for_linear_system(A, b, lr=0.01, max_iter=1000, tol=1e-6):
    """Solve Ax = b using gradient descent on ||Ax - b||^2."""
    n = A.shape[1]
    x = np.zeros(n)
    
    history = {'x': [x.copy()], 'loss': []}
    
    for i in range(max_iter):
        # Loss = ||Ax - b||^2
        residual = A @ x - b
        loss = np.sum(residual ** 2)
        history['loss'].append(loss)
        
        # Gradient = 2 * A^T (Ax - b)
        gradient = 2 * A.T @ residual
        
        # Update
        x = x - lr * gradient
        history['x'].append(x.copy())
        
        if loss < tol:
            print(f"Converged at iteration {i}")
            break
    
    return x, history

# Test
A = np.array([[4, 1], [1, 3]], dtype=float)
b = np.array([1, 2], dtype=float)

x_gd, history = gradient_descent_for_linear_system(A, b, lr=0.1, max_iter=100)
x_exact = np.linalg.solve(A, b)

print(f"\nGradient descent solution: {x_gd}")
print(f"Exact solution: {x_exact}")

# Plot convergence
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curve
axes[0].semilogy(history['loss'])
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Loss (log scale)')
axes[0].set_title('Convergence of Gradient Descent')

# Path in solution space
x_history = np.array(history['x'])
axes[1].plot(x_history[:, 0], x_history[:, 1], 'b.-', alpha=0.5, label='GD path')
axes[1].plot(x_history[0, 0], x_history[0, 1], 'go', markersize=10, label='Start')
axes[1].plot(x_exact[0], x_exact[1], 'r*', markersize=15, label='Exact solution')
axes[1].set_xlabel('x1')
axes[1].set_ylabel('x2')
axes[1].set_title('Gradient Descent Path')
axes[1].legend()

plt.tight_layout()
plt.show()

## 9. Summary

In [ ]:
print("""
SOLVING LINEAR SYSTEMS: SUMMARY
================================

1. EXISTENCE OF SOLUTIONS
   - Unique: rank(A) = rank([A|b]) = n
   - None: rank(A) < rank([A|b])
   - Infinite: rank(A) = rank([A|b]) < n

2. DIRECT METHODS
   - Gaussian elimination: O(n³), general purpose
   - LU decomposition: efficient for same A, different b

3. LEAST SQUARES
   - When no exact solution exists (overdetermined)
   - Normal equations: A^T A x = A^T b
   - Geometric: projection onto column space

4. NUMERICAL CONSIDERATIONS
   - Condition number: measures sensitivity
   - Ridge regression: adds λI to stabilize

5. ITERATIVE METHODS
   - Gradient descent: simple but can be slow
   - Conjugate gradient: optimal for symmetric positive definite
   - Essential for very large systems
""")